In [1]:
# Test this with Llama 3.2 3B and 1B models

import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [12]:
import requests

# Define the endpoint and model
url = "http://localhost:1234/v1/completions"
model = "hugging-quants/Llama-3.2-1B-Instruct-Q8_0-GGUF"


# NOTE: that trying to add the tool calling description as a system prompt does NOT work
# but directly adding it to the query did. 
query = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Questions: Can you tell me the weather for Cape Coral? 

Here is a list of functions in JSON format that you can invoke:
[
    {
        "name": "turn_on_light",
        "description": "Send an HTTP request to turn on a smart lightbulb in a specific room.",
        "parameters": {
            "type": "dict",
            "required": [
                "room"
            ],
            "properties": {
                "room": {
                    "type": "string",
                    "description": "The room where the light is located. Must be one of: living room, kitchen, office, bedroom, or garage.",
                    "enum": ["livingroom", "kitchen", "office", "bedroom", "garage"]
                }
            }
        }
    },

    {
        "name": "fetch_weather",
        "description": "Fetch the current weather for a given location.",
        "parameters": {
            "type": "dict",
            "required": [
                "location"
            ],
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the city or coordinates (latitude,longitude) to fetch the weather for."
                },
                "units": {
                    "type": "string",
                    "description": "The unit system for temperature (metric for Celsius, imperial for Fahrenheit).",
                    "enum": ["metric", "imperial"],
                    "default": "metric"
                }
            }
        }
    },
    {
        "name": "set_alarm",
        "description": "Set an alarm for a specific time.",
        "parameters": {
            "type": "dict",
            "required": [
                "time"
            ],
            "properties": {
                "time": {
                    "type": "string",
                    "description": "The time to set the alarm for, in HH:MM 24-hour format."
                },
                "label": {
                    "type": "string",
                    "description": "An optional label for the alarm.",
                    "default": "Alarm"
                }
            }
        }
    },
    
]

Should you decide to return the function call, Put it in the format of func(params_name=params_value, params_name2=params_value2...)
NO other text MUST be included.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
# Define the payload for the API request
data = {
    "model": model,
    "prompt": query,
}

# Make the API request
response = requests.post(url, json=data)

# Check if the response was successful
if response.status_code == 200:
    result = response.json()
    print(result['choices'][0]['text'].strip())  # Extract and print the response text
else:
    print(f"Error: {response.status_code}, {response.text}")


"fetch_weather(location="Cape Coral", units="metric")
